## BUILDING A HACKER NEWS PIPELINE

### OVERVIEW
In this guided project, we will build a pipeline and apply it to a real world data pipeline project. From a JSON API, we will filter, clean, aggregate, and summarize data in a sequence of tasks that will apply these transformations for us.

The data we will use comes from a Hacker News (HN) API that returns JSON data of the top stories in 2014. If you're unfamiliar with Hacker News, it's a link aggregator website that users vote up stories that are interesting to the community. It is similar to Reddit, but the community only revolves around on computer science and entrepreneurship posts.

 The JSON file contains a single key stories, which contains a list of stories (posts). Each post has a set of keys, but we will deal only with the following keys:
 
 
 * created_at: A timestamp of the story's creation time.
 * created_at_i: A unix epoch timestamp.
 * url: The URL of the story link.
 * objectID: The ID of the story.
 * author: The story's author (username on HN).
 * points: The number of upvotes the story had.
 * title: The headline of the post.
 * num_comments: The number of a comments a post has.
 
 ## AIM
Using this dataset, we will run a sequence of basic natural language processing tasks using our Pipeline class. The goal will be to find the top 100 keywords of Hacker News posts in 2014. Because Hacker News is the most popular technology social media site, this will give us an understanding of the most talked about tech topics in 2014!


In [9]:
# Import the Pipeline class from the pipeline module
from datetime import datetime
import json
import io
import string

from pipeline import build_csv, Pipeline
from stop_words import stop_words




#Instantiate an instance of the Pipeline class and assign it to the variable
pipeline = Pipeline()

## Loading the JSON Data

Since JSON files resemble a key-value dictionary, the goal is to paarse the JSON file into a python dict object using the json module.

To load in a file, json exposes a method called json.load() which takes in a Python file object as the first argument. Using this json.load() method, we'll load the hn_stories_2014.json file as a Python dict.

In [10]:
# Create a pipeline.task function with no arguement
# Call a function that loads the json file in to a python dict and returns the sotories
pipeline.task()
def file_to_json():
    with open('hn_stories_2014.json', 'r') as f:
        data = json.load(f)
        stories = data['stories']
    return stories

## Filtering the Stories

Now that we have loaded in all the stories as a list of dict objects, we can now operate on them. Let's start by filtering the list of stories to get the most popular stories of the year.

Like any social link aggregator site, individual users can post whatever content they want. The reason we want the most popular stories is to ensure that we select stories that were the most talked about during the year. We can filter for popular stories by ensuring they are links (not Ask HN posts), have a good number of points, and have some comments.

# Do:

* Create a pipeline task that depends on the file_to_json() function
* Call the new function filter_stories(), that filters popular stories that have more than 50 points, more than 1 comment, and do not begin with Ask HN
* filter_stories() should return a generator of these filtered stories.

In [11]:
pipeline.task(depends_on=file_to_json)
def filter_stories(stories):
    def is_popular(story):
        return story['points'] > 50 and story['num_comments'] > 1 and not story['title'].startswith('Ask HN')
    
    return (
        story for story in stories
        if is_popular(story)
    )

## Convert to CSV

With a reduced set of stories, it's time to write these dict objects to a CSV file. The purpose of translating the dictionaries to a CSV is that we want to have a consistent data format when running the later summarizations. By keeping consistent data formats, each of your pipeline tasks will be adaptable with future task requirements.

# Do:

* Create a pipeline.task() function that depends on the filter_stories() function.
* Call the new function json_to_csv(), that writes the filtered JSON stories to a CSV file:
  * Import build_csv from the pipelines module and io. The build_csv() function has the same API as the one you wrote in the second and third mission.
  * eate a CSV file with the headers 'objectID', 'created_at', 'url', 'points', and 'title'.
  * rse the created_at column using datetime.datetime.
* on_to_csv() should return the value from build_csv() using the above header, lines, and the io.StringIO() file.




In [17]:
@pipeline.task(depends_on=filter_stories)
def json_to_csv(stories):
    lines = []
    for story in stories:
        lines.append(
            (story['objectID'], datetime.strptime(story['created_at'], "%Y-%m-%dT%H:%M:%SZ"), story['url'], story['points'], story['title'])
        )
    return build_csv(lines, header=['objectID', 'created_at', 'url', 'points', 'title'], file=io.StringIO())


## Extract Title Column

Using the CSV file format we created in the previous task, we can now extract the title column. Once we have extracted the titles of each popular post, we can then run the next word frequency task. To extract the titles, we'll follow the steps:

1. Import csv, and create a csv.reader() object from the file object.

2. Find the index of the title in the header.

3. Iterate the through the reader, and return each item from the reader in the corresponding title index position.

In [18]:
import csv
@pipeline.task(depends_on=json_to_csv)
def extract_titles(csv_file):
    reader = csv.reader(csv_file)
    header = next(reader)
    idx = header.index('title')
    
    return (line[idx] for line in reader)

## Clean the Titles

Because we're trying to create a word frequency model of words from Hacker News titles, we need a way to create a consistent set of words to use. For example, words like Google, google, GooGle?, and google., all mean the same keyword: google. If we were to split the title into words, however, they would all be lumped into different categories.

To clean the titles, we should make sure to lower case the titles, and to remove the punctuation. An easy way to rid a string of punctuation is to check each character, determine if it is a letter or punctuation, and only keep the letter. From the string package, we are given a handy string constant that contains all the punctuation needed:


In [19]:
@pipeline.task(depends_on=extract_titles)
def clean_title(titles):
    for title in titles:
        title = title.lower()
        title = ''.join(c for c in title if c not in string.punctuation)
        yield title

## Create the Word Frequency Dictionary

With a cleaned title, we can now build the word frequency dictionary. A word frequency dictionary are key value pairs that connects a word to the number of times it is used in a text.

As you can see, the title has been stripped of its punctuation and lower cased. Furthermore, to find actual keywords, we should enforce the word frequency dictionary to not include stop words.

Stop words are words that occur frequently in language like "the", "or", etc., and are commonly rejected in keyword searches.

We have included a module called stop_words with a tuple of the most common used stop words in the English language.

In [20]:
@pipeline.task(depends_on=clean_title)
def build_keyword_dictionary(titles):
    word_freq = {}
    for title in titles:
        for word in title.split(' '):
            if word and word not in stop_words:
                if word not in word_freq:
                    word_freq[word] = 1
                word_freq[word] += 1
    return word_freq

## Sort the Top Words

Finally, we're ready to sort the top words used in all the titles. In this final task, it's up to you to decide how you want to sort the top words. The goal is to output a list of tuples with (word, frequency) as the entries sorted from most used, to least most used.

In [21]:
@pipeline.task(depends_on=build_keyword_dictionary)
def top_keywords(word_freq):
    freq_tuple = [
        (word, word_freq[word])
        for word in sorted(word_freq, key=word_freq.get, reverse=True)
    ]
    return freq_tuple[:100]

ran = pipeline.run()
print(ran[top_keywords])

TypeError: filter_stories() missing 1 required positional argument: 'stories'